In [1]:
import re
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
import warnings 
warnings.filterwarnings('ignore')

C:\Users\epile\anaconda3\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
import re
import numpy as np
import pandas as pd

# Función para detectar y sumar números en un rango específico en una cadena de texto
def sumar_numeros_rango(df, columna_texto):
    def extraer_y_sumar(text):
        # Encuentra todos los números en la cadena
        numeros = re.findall(r'\d+(?:\.\d{1,2})?', text)
        # Filtra los números dentro del rango 50-50000, conviértelos a float y sume
        numeros_en_rango = [float(num) for num in numeros if 50 <= float(num) <= 50000]
        # Si hay números en el rango, devuelve la suma; si no, devuelve NaN
        return sum(numeros_en_rango) if numeros_en_rango else np.nan

    # Aplicar la función a la columna especificada y guardar el resultado en una nueva columna 'Ingreso'
    df['Ingreso'] = df[columna_texto].apply(lambda x: extraer_y_sumar(str(x)))

data = pd.read_csv('data/REPARACIONES.csv', encoding='latin1')
data[['CODIGO', 'INFORMETALLER']]

,CODIGO,INFORMETALLER
0,1,CAMBIO DEL PANEL DE CONTROL $380+IVA BUEN MANT...
1,2,"FUENTE DAÑADA, FUSOR DAÑADO/INCOSTIABLE/ BASUR..."
2,2069,INCLUIR EL EDITOR DEL PDF $150+IVA
3,3,MANTENIMIENTO $480+IVA
4,28,"fusor completo. rodillo de calor dañado, funda..."
...,...,...
11945,11957,NaN
11946,11939,MANTENIMIENTO EXHAUSTIVO $905 NETO \r\n\r\n\r\...
11947,11940,*REQUIERE DE CAMBIO DE GOMA DE TRACCIÓN $380 N...
11948,11941,CAMBIO DE RODILLO DE CALOR $680 NETO\r\nMANTEN...


In [3]:
sumar_numeros_rango(data, 'INFORMETALLER')
data[['CODIGO', 'INFORMETALLER', 'Ingreso']]

,CODIGO,INFORMETALLER,Ingreso
0,1,CAMBIO DEL PANEL DE CONTROL $380+IVA BUEN MANT...,1820.0
1,2,"FUENTE DAÑADA, FUSOR DAÑADO/INCOSTIABLE/ BASUR...",NaN
2,2069,INCLUIR EL EDITOR DEL PDF $150+IVA,150.0
3,3,MANTENIMIENTO $480+IVA,480.0
4,28,"fusor completo. rodillo de calor dañado, funda...",NaN
...,...,...,...
11945,11957,NaN,NaN
11946,11939,MANTENIMIENTO EXHAUSTIVO $905 NETO \r\n\r\n\r\...,905.0
11947,11940,*REQUIERE DE CAMBIO DE GOMA DE TRACCIÓN $380 N...,2120.0
11948,11941,CAMBIO DE RODILLO DE CALOR $680 NETO\r\nMANTEN...,4218.0


In [4]:
data_cleaned = data.dropna(subset=['Ingreso'])
data_cleaned[['INFORMETALLER', 'Ingreso']]

,INFORMETALLER,Ingreso
0,CAMBIO DEL PANEL DE CONTROL $380+IVA BUEN MANT...,1820.0
2,INCLUIR EL EDITOR DEL PDF $150+IVA,150.0
3,MANTENIMIENTO $480+IVA,480.0
5,rep. de mylar de teclado $ 360.00 + iva,360.0
6,REPARACION DE MYLER DE TECLADO $ 360.00 + IVA,360.0
...,...,...
11941,*MANTENIMIENTO Y REPARACIÓN DE MEMBRANA DE TEC...,880.0
11944,RESORTE DE SENSOR DE PAPEL $400+IVA--ACEPTADO\...,3695.0
11946,MANTENIMIENTO EXHAUSTIVO $905 NETO \r\n\r\n\r\...,905.0
11947,*REQUIERE DE CAMBIO DE GOMA DE TRACCIÓN $380 N...,2120.0


In [5]:
data_cleaned['FALLA_STR'] = str(data_cleaned['FALLA'])
data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9375 entries, 0 to 11948
Data columns (total 49 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   CODIGO               9375 non-null   int64  
 1   CLIENTE              9375 non-null   int64  
 2   NS                   9374 non-null   object 
 3   FALLA                8959 non-null   object 
 4   OBSERVACIONES        4847 non-null   object 
 5   PROMETIDO            9375 non-null   object 
 6   FECHAINGRESO         9375 non-null   object 
 7   HORAINGRESO          9375 non-null   object 
 8   RECEPCIONADO         9375 non-null   int64  
 9   TECNICO              9375 non-null   int64  
 10  INFORMETALLER        9375 non-null   object 
 11  FECHATERMINADO       9375 non-null   object 
 12  HORATERMINADO        9172 non-null   object 
 13  ESTADO               9375 non-null   int64  
 14  ENTREGADO            9375 non-null   object 
 15  AVISADO              9375 non-null   

In [6]:
keywords = {
    'Impresora Laser': ['impresora laser', 'tn', 'toner', 'drum',
                        'calor', 'presion', 'pad', 'termistor'],
    'Impresora Tinta': ['impresora tinta', 'reset', 'almohadillas', 'bateria',
                        '12v', 'cinta', 'perilla', 'cabezal', 'manguera', 'mascarilla'],
    'Sistemas': ['sistema', 'office', 'pantalla', 'windows', 'bisagra', 'teclado',
                 'mouse', 'mousepad', 'ssd', 'operativo', 'disco duro',
                 'ram', 'procesador', 'madre', 'correo', 'gabinete', 'bios'],
    'Otros': ['otros']
}

In [7]:
def get_embedding(texts):
    EMBEDDING_MODEL = "all-MiniLM-L6-v2"
    model = SentenceTransformer(EMBEDDING_MODEL)
    embedding = model.encode(texts, convert_to_numpy=True) 
    return embedding 

def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_a = np.linalg.norm(vec1)
    norm_b = np.linalg.norm(vec2)
    return dot_product / (norm_a * norm_b)

In [8]:
data.columns

Index(['CODIGO', 'CLIENTE', 'NS', 'FALLA', 'OBSERVACIONES', 'PROMETIDO',
       'FECHAINGRESO', 'HORAINGRESO', 'RECEPCIONADO', 'TECNICO',
       'INFORMETALLER', 'FECHATERMINADO', 'HORATERMINADO', 'ESTADO',
       'ENTREGADO', 'AVISADO', 'BATERIA', 'BATERIA_ORIG', 'CARGADOR',
       'CARGADOR_ORIG', 'PROMETIDO_HORA', 'CHIP', 'CHIP_NS', 'ORDEN_CLI',
       'CLI_NOMBRE_EVENTUAL', 'TEL_EVENTUAL', 'CLI_EVENTUAL',
       'FECHA_ENTREGADO', 'CLIENTE_GARANTIA', 'SUCURSALCLI', 'PIN',
       'CAMPOLIBRE1', 'CAMPOLIBRE2', 'GARANTIA', 'FECHA_AUD', 'HORA_AUD',
       'GARANTIA_FECHA', 'GARANTIA_FACTURA', 'SITUACION', 'TAREA',
       'UBICACION_ENTRADA', 'UBICACION_SALIDA', 'REMITO_ENTREGADO',
       'REMITO_INGRESO', 'BARCODE', 'ARTICULO', 'AREAVENTA', 'Ingreso'],
      dtype='object')

In [78]:
imp_laser = []
imp_tinta = []
sistemas = []
otros = []

for palabra in keywords['Impresora Laser']:
    embedding = get_embedding(palabra)
    imp_laser.append(embedding)

for palabra in keywords['Impresora Tinta']:
    embedding = get_embedding(palabra)
    imp_tinta.append(embedding)

for palabra in keywords['Sistemas']:
    embedding = get_embedding(palabra)
    sistemas.append(embedding)

for palabra in keywords['Otros']:
    embedding = get_embedding(palabra)
    otros.append(embedding)

In [79]:
keywords_embedding = {
    'Impresora Laser':imp_laser,
    'Impresora Tinta':imp_tinta,
    'Sistemas':sistemas,
    'Otros':otros
}

In [80]:
data_cleaned["COMBINADO"] = (
    data_cleaned["FALLA"].fillna(" ") + " " +
    data_cleaned["OBSERVACIONES"].fillna(" ") + " " +
    data_cleaned["INFORMETALLER"].fillna(" ")
)

In [85]:
muestra_embedding = []
muestra = data_cleaned['COMBINADO'].head(10).reset_index()

for i in range(len(muestra)):
    embedding = get_embedding(muestra['COMBINADO'][i])
    muestra_embedding.append(embedding)

In [94]:
#### Lista para almacenar los resultados
resultados_correlacion = []

# Calcular la correlación de cada vector en muestra_embedding con cada vector en cada lista del diccionario
for i, vector in enumerate(muestra_embedding):
    for key, lista_vectores in keywords_embedding.items():
        for j, vector_dict in enumerate(lista_vectores):
            cos_sim = cosine_similarity(vector, vector_dict)
            resultados_correlacion.append({
                "Número de falla": i + 1, 
                "Segmento": key,  
                "Palabra": j + 1, 
                "Similitud Coseno": cos_sim
            })

df_resultados = pd.DataFrame(resultados_correlacion)

df_resultados

,Número de falla,Segmento,Palabra,Similitud Coseno
0,1,Impresora Laser,1,0.143885
1,1,Impresora Laser,2,0.171205
2,1,Impresora Laser,3,0.099955
3,1,Impresora Laser,4,-0.000467
4,1,Impresora Laser,5,0.112204
...,...,...,...,...
355,10,Sistemas,14,0.134226
356,10,Sistemas,15,0.273986
357,10,Sistemas,16,0.190798
358,10,Sistemas,17,0.019422
